In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
#project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
#sys.path.insert(0, project_root)
sys.path.append("..") 
sys.path.append("../src")
from src.load_data import  prepare_volatility_data
from src.load_data import load_pivoted_field
from src.forecast import build_forecast_pipeline
from src.signals import generate_signals_and_positions
from src.analytic_utils import align_multiple

In [3]:
from src.checkpoint_storage import save_all_checkpoints_v2,load_all_checkpoints_v2

In [4]:
checkpoint=load_all_checkpoints_v2("../checkpoints/har_pre_model_20250804_140707")
print(type(checkpoint), checkpoint.keys() if isinstance(checkpoint, dict) else None)
ds_test=checkpoint["test"]

print(type(ds_test), ds_test.keys() if isinstance(ds_test, dict) else None)
print(checkpoint["train"]["X_df"].columns.tolist())

best_model=checkpoint["model"]
print(best_model.feature_names_in_)

<class 'dict'> dict_keys(['train', 'val', 'test', 'model', 'metrics', 'config', 'notes'])
<class 'dict'> dict_keys(['X_df', 'y_series', 'label_matrix', 'idx', 'features'])
['signal', 'signal_rank', 'implied', 'implied_rank']
['signal' 'signal_rank' 'implied' 'implied_rank']


In [5]:
idx_df=ds_test['idx']  
multi_idx = pd.MultiIndex.from_frame(idx_df, names=["timestamp", "ticker"])
multi_idx

MultiIndex([('2023-11-20',  'AAL'),
            ('2023-11-20',  'AAP'),
            ('2023-11-20', 'AAPL'),
            ('2023-11-20', 'ABBV'),
            ('2023-11-20',  'ABT'),
            ('2023-11-20',  'ACN'),
            ('2023-11-20', 'ADBE'),
            ('2023-11-20',  'ADM'),
            ('2023-11-20',  'ADP'),
            ('2023-11-20',  'AEP'),
            ...
            ('2023-12-29',  'WRB'),
            ('2023-12-29',  'WRK'),
            ('2023-12-29',  'WST'),
            ('2023-12-29',   'WY'),
            ('2023-12-29', 'WYNN'),
            ('2023-12-29',  'XOM'),
            ('2023-12-29',  'ZBH'),
            ('2023-12-29', 'ZBRA'),
            ('2023-12-29', 'ZION'),
            ('2023-12-29',  'ZTS')],
           names=['timestamp', 'ticker'], length=11020)

In [6]:


X_test=checkpoint["test"]["X_df"]
y_test=checkpoint["test"]["y_series"]
y_pred=best_model.predict_proba(X_test)[:,1]
y_pred_binary=np.where(y_pred>0.5,1,0)
y_test_binary=np.where(y_test>0.5,1,0)
accuracy=roc_auc_score(y_test_binary,y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7474987841253247


In [7]:
y_pred_series = pd.Series(y_pred, index=multi_idx)
y_pred_series
y_pred_matrix=y_pred_series.unstack()
y_pred_matrix
ranked = y_pred_matrix.rank(axis=1, pct=True)
ranked
buy_mask = ranked >= 0.9
sell_mask = ranked <= 0.1
buy_mask
trades = []

for dt in y_pred_matrix.index:
    for ticker in y_pred_matrix.columns:
        action = None
        if buy_mask.loc[dt, ticker]:
            action = "buy"
        elif sell_mask.loc[dt, ticker]:
            action = "sell"
        if action:
            trades.append((dt, ticker, action))

In [8]:
import json
import yaml

with open("../configs/config.json") as f:
    config_data = json.load(f)

with open("../configs/config.yaml") as f:
    config_exp = yaml.safe_load(f)

print(config_data)
print(config_exp)
vol_data = prepare_volatility_data(config_data)



{'realized_path': 'data/iv_data.parquet', 'implied_path': 'data/iv_implied.parquet', 'price_path': 'data/underlying_prices.parquet', 'forecast_horizon': 5, 'model_path': 'models/best_model.pkl', 'new_realized_path': 'data/new_realized.parquet', 'new_implied_path': 'data/new_implied.parquet', 'option_chain_path': 'data/option_chain.parquet', 'output_ledger_path': 'results/ledger.parquet', 'train_test_split_ratio': 0.7, 'train_test_gap_days': 21, 'ranking_threshold': 0.9, 'signal_z': True, 'implied_z': True, 'rf_params': {'n_estimators': 100, 'max_depth': 5}, 'har_windows': {'short': 1, 'medium': 5, 'long': 21}}
{'experiment_name': 'v2_rf_20pct', 'signal_threshold': 0.2, 'features': ['signal', 'signal_rank', 'implied', 'implied_rank'], 'model': {'type': 'random_forest', 'hyperparams': {'n_estimators': 100, 'max_depth': 5}}}


/Users/stirlitz/ncGitHub/vol_forecast/notebooks/../src/load_data.py:149: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_returns = df_prices.pct_change()


In [10]:
forecast = build_forecast_pipeline(vol_data["realized"], vol_data["earnings_mask"], config_data)

In [17]:
close_matrix=load_pivoted_field("close")
close_matrix.index.rename("date", inplace=True)
close_matrix.head()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,156.420,18.7400,236.76,182.000,135.42,132.65,139.04,44.550,407.20,564.37,...,87.75,67.95,63.545,56.900,116.90,136.56,129.13,583.89,64.270,234.01
2022-01-04,151.200,19.0200,237.16,179.690,135.14,131.40,135.76,45.130,404.48,554.10,...,87.49,68.37,65.910,57.190,118.10,137.31,130.44,587.06,66.055,225.04
2022-01-05,148.680,18.6900,236.45,174.835,135.88,132.50,135.17,44.595,397.25,514.28,...,84.25,69.05,66.740,56.270,116.57,135.58,130.01,558.65,66.200,216.65
2022-01-06,149.110,18.5653,241.59,172.010,135.20,130.45,135.14,44.855,378.01,514.05,...,84.64,68.52,68.310,56.210,116.36,136.90,128.92,555.16,68.930,217.46
2022-01-07,145.142,19.2900,238.09,172.220,134.88,133.12,135.54,45.080,370.75,510.63,...,84.53,69.12,68.880,56.405,115.39,135.37,128.14,530.57,70.080,211.09


In [18]:
def get_daily_snapshot(
    date: pd.Timestamp,
    close_matrix: pd.DataFrame,
    iv_matrix: pd.DataFrame
) -> pd.DataFrame:
    """
    Return a DataFrame with 'ticker', 'close', 'implied_vol' for a given date.
    """
    close_row = close_matrix.loc[date]
    iv_row = iv_matrix.loc[date]

    common_tickers = close_row.index.intersection(iv_row.index)

    df = pd.DataFrame({
        "ticker": common_tickers,
        "close": close_row[common_tickers].values,
        "implied_vol": iv_row[common_tickers].values
    }).dropna()

    return df

df=get_daily_snapshot(pd.Timestamp("2022-07-28"), close_matrix, vol_data["implied"])

In [20]:
import numpy as np
import pandas as pd
import py_vollib.black_scholes
import py_vollib_vectorized



def compute_option_price(spot, strike, cp, ttm, r, q, sigma):
    """
    Vectorized option pricing using py_vollib_vectorized.black_scholes.
    
    Parameters:
        spot: float or array
        strike: float or array
        cp: 'call' or 'put' or array-like
        ttm: float or array (in years)
        r: risk-free rate (scalar or array)
        q: dividend yield (scalar or array)
        sigma: implied volatility (float or array)
    
    Returns:
        np.ndarray of option prices
    """
    cp = np.asarray(cp)
    print(f"cp: {cp}")
    cp = np.where(cp == 'call', 'c', 'p')  # convert to py_vollib format
    print(f"cp: {cp}")

    return py_vollib.black_scholes.black_scholes(
        flag=cp,
        S=np.asarray(spot),
        K=np.asarray(strike),
        t=np.asarray(ttm),
        r=np.asarray(r),
        sigma=np.asarray(sigma)
    )

In [81]:
ledger=TradeLedger()
expiry=pd.Timestamp("2025-07-31")

In [14]:
for dt, ticker, action in trades:
    snapshot = load_option_snapshot(ticker, dt)
    spot = load_underlying_price(ticker, dt)
    atm_strike = snapshot["strike"].sub(spot).abs().idxmin()

    for opt_type in ["call", "put"]:
        row = snapshot.loc[
            (snapshot["strike"] == snapshot.loc[atm_strike, "strike"]) &
            (snapshot["option_type"] == opt_type)
        ]
        if row.empty: continue

        ledger.record_trade(
            timestamp=dt,
            action=action,
            instrument_type="option",
            underlying=ticker,
            option_type=opt_type,
            strike=row["strike"].values[0],
            expiry=row["expiry"].values[0],
            quantity=1,
            price=row["mid_price"].values[0]
        )

NameError: name 'load_option_snapshot' is not defined

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
from zoneinfo import ZoneInfo
from typing import Callable

def compute_payoff(spot, strike, opt_type):
    if opt_type == "call":
        return np.maximum(spot - strike, 0)
    else:

        return np.maximum(strike - spot, 0)

def compute_payoff_vec(spots: np.ndarray,
                       strikes: np.ndarray,
                       opt_types: np.ndarray) -> np.ndarray:
    """
    spots, strikes: float arrays of the same shape
    opt_types: array of 'call' or 'put' strings, same shape
    returns: array of intrinsic payoffs
    """
    # mask of where it's a call
    is_call = (opt_types == "call")
    # intrinsic payoff for calls and for puts
    call_payoff = np.maximum(spots - strikes, 0)
    put_payoff  = np.maximum(strikes - spots, 0)
    # pick the right one for each element
    return np.where(is_call, call_payoff, put_payoff)

class TradeLedger:
    def __init__(self):
        cols = [
            "timestamp","action","instrument_type","underlying",
            "option_type","strike","expiry","quantity","price",
            "signed_quantity","total_cost"
        ]
        self.trades_df = pd.DataFrame(columns=cols)

    def record_trades(self, new_trades: pd.DataFrame):
        """Batch append a DataFrame of trades."""
        print(f"new_trades: {new_trades}")
        self.trades_df = pd.concat([self.trades_df, new_trades], ignore_index=True)
        print(f"self.trades_df: {self.trades_df}")

    def record_trade(self,
                     timestamp: datetime,
                     action: str,
                     instrument_type: str,
                     underlying: str,
                     option_type: str,
                     strike: float,
                     expiry: date,
                     quantity: float,
                     price: float):
        """
        Add a single trade.
        Automatically computes signed_quantity and total_cost.
        """
        signed_qty = quantity if action == "buy" else -quantity
        row = {
            "timestamp": timestamp,
            "action": action,
            "instrument_type": instrument_type,
            "underlying": underlying,
            "option_type": option_type,
            "strike": strike,
            "expiry": expiry,
            "quantity": quantity,
            "price": price,
            "signed_quantity": signed_qty,
            "total_cost": signed_qty * price
        }
        # create 1-row DataFrame and concat
        self.trades_df = pd.concat(
            [self.trades_df, pd.DataFrame([row])],
            ignore_index=True
        )

    @property
    def trades(self):
        """All trades, sorted by timestamp."""
        return self.trades_df.sort_values("timestamp").reset_index(drop=True)

class PortfolioView:
    def __init__(self, ledger: TradeLedger):
        self.ledger = ledger

    def positions_at(self, as_of: datetime) -> pd.DataFrame:
        df = self.ledger.trades_df
        print(f"trades_df: {df}")
        print (f"as_of: {as_of}")
        print (f"df[\"timestamp\"]: {df["timestamp"]}")
        df = df[df["timestamp"] <= as_of]
        print("as_of: ", as_of)
        print(f"df after filtering: {df}")
        grouped = (
            df.groupby(
                ["instrument_type","underlying","option_type","strike","expiry"]
            )["signed_quantity"]
            .sum()
            .reset_index()
        )
        return grouped[grouped["signed_quantity"] != 0]

def expire_trades_vectorized(
    ledger: TradeLedger,
    as_of_date: date,
    price_lookup_fn: Callable[[np.ndarray, date], np.ndarray]
):
    close_dt = datetime.combine(
        as_of_date, time(16,30), tzinfo=ZoneInfo("America/New_York")
    )
    opens = PortfolioView(ledger).positions_at(close_dt)
    opens["expiry_date"] = pd.to_datetime(opens["expiry"]).dt.date
    print (f"runnning option expiry, as_of_date: {as_of_date}, close_dt: {close_dt}")
    print (f"opens: {opens}")
    to_close = opens[pd.to_datetime(opens["expiry_date"]) <= pd.to_datetime(as_of_date).normalize()]
    if to_close.empty:
        return

    spots = price_lookup_fn(to_close["underlying"].values, as_of_date)
    print(f"spots: {spots}")
    payoffs = compute_payoff_vec(
        spots,
        to_close["strike"].values,
        to_close["option_type"].values
    )
    actions = np.where(to_close["signed_quantity"] > 0, "sell", "buy")
    qtys    = to_close["signed_quantity"].abs().values

    batch = pd.DataFrame({
        "timestamp":      close_dt,
        "action":         actions,
        "instrument_type":"option",
        "underlying":     to_close["underlying"].values,
        "option_type":    to_close["option_type"].values,
        "strike":         to_close["strike"].values,
        "expiry":         to_close["expiry"].values,
        "quantity":       qtys,
        "price":          payoffs,
    })
    batch["signed_quantity"] = np.where(batch["action"]=="buy",
                                        batch["quantity"],
                                       -batch["quantity"])
    batch["total_cost"] = batch["signed_quantity"] * batch["price"]
    print (f"batch: {batch}")

    ledger.record_trades(batch)


ledger = TradeLedger()
#expire_trades_vectorized(ledger, date(2025, 7, 31), price_lookup_fn)
#ledger.trades

In [23]:
import numpy as np
import pandas as pd
import py_vollib.black_scholes
import py_vollib_vectorized



def compute_option_price(spot, strike, cp, ttm, r, q, sigma):
    """
    Vectorized option pricing using py_vollib_vectorized.black_scholes.
    
    Parameters:
        spot: float or array
        strike: float or array
        cp: 'call' or 'put' or array-like
        ttm: float or array (in years)
        r: risk-free rate (scalar or array)
        q: dividend yield (scalar or array)
        sigma: implied volatility (float or array)
    
    Returns:
        np.ndarray of option prices
    """
    cp = np.asarray(cp)
    cp = np.where(cp == 'call', 'c', 'p')  # convert to py_vollib format

    print(f"spot: {spot}")
    print(f"strike: {strike}")
    print(f"cp: {cp}")
    print(f"ttm: {ttm}")
    print(f"r: {r}")
    print(f"q: {q}")
    print(f"sigma: {sigma}")
    return py_vollib.black_scholes.black_scholes(
        flag=cp,
        S=np.asarray(spot),
        K=np.asarray(strike),
        t=np.asarray(ttm),
        r=np.asarray(r),
        sigma=np.asarray(sigma),
        return_as='numpy'
    )

In [24]:
trades
def expand_and_record_straddle_trades(trades, ledger, close_matrix, iv_matrix):
    print(f'trades: {trades}')
    vols= iv_matrix.loc[trades["timestamp"], trades["underlying"]]
    spots= close_matrix.loc[trades["timestamp"], trades["underlying"]]

    strike = round(spot, 1)  # optional: round to nearest 0.5 or 1
    expiry = dt + timedelta(days=1)
    ttm = 1/252  # or actual days to expiry

    for opt_type in ['call', 'put']:
        price = compute_option_price(
            spot=spot,
            strike=strike,
            cp=opt_type,
            ttm=ttm,
            r=0.0,
            q=0.0,
            sigma=vols
        )

        ledger.record_trade(
            timestamp=dt,
            action=action,
            instrument_type="option",
            underlying=ticker,
            option_type=opt_type,
            strike=strike,
            expiry=expiry.date(),
            quantity=1,
            price=price
        )


In [25]:
from datetime import timedelta

def generate_straddle_trades(
    trades,              # List of (timestamp, ticker, action)
    close_matrix,        # DataFrame: [date x ticker] of spot prices
    vol_matrix,          # DataFrame: [date x ticker] of implied vols
    ledger,              # Your TradeLedger instance
    ttm_days=5,          # Days to expiry
    r=0.0,
    q=0.0
):
    trade_dates = [t[0] for t in trades]
    time_stamps = pd.to_datetime(trade_dates).tz_localize("America/New_York")
    #print (f"trade_dates: {trade_dates}")
    tickers = [t[1] for t in trades]
    actions = [t[2] for t in trades]
    notionals = np.array([1 if a == "buy" else -1 for a in actions])

    spots = np.array([
        close_matrix.loc[dt, ticker] for dt, ticker in zip(trade_dates, tickers)
    ])
    vols = np.array([
        vol_matrix.loc[dt, ticker] for dt, ticker in zip(trade_dates, tickers)
    ])
    strike = spots
    qty = 1 / spots
    signed_qty = qty * notionals
    #print (f"about to compute expiry")
    expiry = [dt + timedelta(days=ttm_days) for dt in trade_dates]
    #print (f"expiry: {expiry}")
    ttm = np.full(len(spots), ttm_days / 252)
    r_arr = np.full(len(spots), r)
    q_arr = np.full(len(spots), q)

    for opt_type in ["call", "put"]:
        price = compute_option_price(
            spot=spots,
            strike=strike,
            cp=[opt_type] * len(spots),
            ttm=ttm,
            r=r_arr,
            q=q_arr,
            sigma=vols
        )

        trades_df = pd.DataFrame({
            "timestamp": time_stamps,
            "action": actions,
            "instrument_type": ["option"] * len(trade_dates),
            "underlying": tickers,
            "option_type": [opt_type] * len(trade_dates),
            "strike": strike,
            "expiry": expiry,
            "quantity": qty,
            "price": price,
            "signed_quantity": signed_qty,
            "total_cost": price * signed_qty
        })

        ledger.record_trades(trades_df)

In [28]:
ledger.trades

,timestamp,action,instrument_type,underlying,option_type,strike,expiry,quantity,price,signed_quantity,total_cost
0,2023-11-20 00:00:00-05:00,sell,option,AAL,call,12.390,2023-11-25,0.080710,0.222394,-0.080710,-0.017949
1,2023-11-20 00:00:00-05:00,sell,option,CZR,put,46.260,2023-11-25,0.021617,0.933517,-0.021617,-0.020180
2,2023-11-20 00:00:00-05:00,buy,option,DD,put,71.380,2023-11-25,0.014010,0.586937,0.014010,0.008223
3,2023-11-20 00:00:00-05:00,sell,option,DISH,put,3.655,2023-11-25,0.273598,0.167796,-0.273598,-0.045908
4,2023-11-20 00:00:00-05:00,sell,option,DXCM,put,109.010,2023-11-25,0.009173,2.194505,-0.009173,-0.020131
...,...,...,...,...,...,...,...,...,...,...,...
4415,2023-12-29 00:00:00-05:00,buy,option,NDSN,call,264.190,2024-01-03,0.003785,2.321308,0.003785,0.008787
4416,2023-12-29 00:00:00-05:00,buy,option,NTAP,call,88.150,2024-01-03,0.011344,0.784142,0.011344,0.008896
4417,2023-12-29 00:00:00-05:00,sell,option,NTRS,call,84.380,2024-01-03,0.011851,1.527488,-0.011851,-0.018102
4418,2023-12-29 00:00:00-05:00,buy,option,HON,call,209.730,2024-01-03,0.004768,1.567545,0.004768,0.007474


In [27]:

generate_straddle_trades(trades, close_matrix, vol_data["implied"],ledger)

spot: [ 12.39  53.04 101.18 ...  11.4   34.83  43.87]
strike: [ 12.39  53.04 101.18 ...  11.4   34.83  43.87]
cp: ['c' 'c' 'c' ... 'c' 'c' 'c']
ttm: [0.01984127 0.01984127 0.01984127 ... 0.01984127 0.01984127 0.01984127]
r: [0. 0. 0. ... 0. 0. 0.]
q: [0. 0. 0. ... 0. 0. 0.]
sigma: [0.31944319 0.4253202  0.16163496 ... 0.39809594 0.16027979 0.35324214]
new_trades:                      timestamp action instrument_type underlying option_type  \
0    2023-11-20 00:00:00-05:00   sell          option        AAL        call   
1    2023-11-20 00:00:00-05:00   sell          option        AAP        call   
2    2023-11-20 00:00:00-05:00    buy          option        ABT        call   
3    2023-11-20 00:00:00-05:00    buy          option        ADM        call   
4    2023-11-20 00:00:00-05:00    buy          option        ADP        call   
...                        ...    ...             ...        ...         ...   
2205 2023-12-29 00:00:00-05:00    buy          option        WAB        ca

/var/folders/tw/8dy4vcgj7h3fvzr8_jtkzs600000gn/T/ipykernel_1670/3130055747.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.trades_df = pd.concat([self.trades_df, new_trades], ignore_index=True)


In [133]:
ledger.trades

,timestamp,action,instrument_type,underlying,option_type,strike,expiry,quantity,price,signed_quantity,total_cost
0,2023-11-20 00:00:00-05:00,sell,option,AAL,call,12.390,2023-11-25,0.080710,0.222394,-0.080710,-0.017949
1,2023-11-20 00:00:00-05:00,sell,option,CZR,put,46.260,2023-11-25,0.021617,0.933517,-0.021617,-0.020180
2,2023-11-20 00:00:00-05:00,buy,option,DD,put,71.380,2023-11-25,0.014010,0.586937,0.014010,0.008223
3,2023-11-20 00:00:00-05:00,sell,option,DISH,put,3.655,2023-11-25,0.273598,0.167796,-0.273598,-0.045908
4,2023-11-20 00:00:00-05:00,sell,option,DXCM,put,109.010,2023-11-25,0.009173,2.194505,-0.009173,-0.020131
...,...,...,...,...,...,...,...,...,...,...,...
4415,2023-12-29 00:00:00-05:00,buy,option,NDSN,call,264.190,2024-01-03,0.003785,2.321308,0.003785,0.008787
4416,2023-12-29 00:00:00-05:00,buy,option,NTAP,call,88.150,2024-01-03,0.011344,0.784142,0.011344,0.008896
4417,2023-12-29 00:00:00-05:00,sell,option,NTRS,call,84.380,2024-01-03,0.011851,1.527488,-0.011851,-0.018102
4418,2023-12-29 00:00:00-05:00,buy,option,HON,call,209.730,2024-01-03,0.004768,1.567545,0.004768,0.007474


In [30]:
def run_daily_expiry_loop(
    all_dates,             # list of datetime.date
    ledger: TradeLedger,
    close_matrix: pd.DataFrame,
    iv_matrix: pd.DataFrame,
    price_lookup_fn: Callable[[np.ndarray, date], np.ndarray],  # for expire_trades
    #pnl_tracker: PnLTracker
):
    for dt in sorted(all_dates):
        #print(f"dt: {dt}")
        snapshot = get_daily_snapshot(dt, close_matrix, iv_matrix)

        # Step 1: Expire positions that mature today
        print (f"expiring trades for {dt}")
        expire_trades_vectorized(ledger, dt, price_lookup_fn)

        # Step 2: Update daily PnL
        #pnl_tracker.update(dt, ledger, snapshot)

In [138]:
ledger.trades[ledger.trades["underlying"]=="AAPL"]

,timestamp,action,instrument_type,underlying,option_type,strike,expiry,quantity,price,signed_quantity,total_cost
2630,2023-12-05 00:00:00-05:00,buy,option,AAPL,put,193.50,2023-12-10,0.005168,1.977090,0.005168,0.010218
2670,2023-12-05 00:00:00-05:00,buy,option,AAPL,call,193.50,2023-12-10,0.005168,1.977090,0.005168,0.010218
2927,2023-12-06 00:00:00-05:00,buy,option,AAPL,put,192.33,2023-12-11,0.005199,1.866290,0.005199,0.009704
3016,2023-12-06 00:00:00-05:00,buy,option,AAPL,call,192.33,2023-12-11,0.005199,1.866290,0.005199,0.009704
3307,2023-12-07 00:00:00-05:00,buy,option,AAPL,put,194.24,2023-12-12,0.005148,1.863480,0.005148,0.009594
3309,2023-12-07 00:00:00-05:00,buy,option,AAPL,call,194.24,2023-12-12,0.005148,1.863480,0.005148,0.009594
3740,2023-12-11 16:30:00-05:00,sell,option,AAPL,call,192.33,2023-12-11,0.005199,0.860000,-0.005199,-0.004471
3741,2023-12-11 16:30:00-05:00,sell,option,AAPL,call,193.50,2023-12-10,0.005168,0.000000,-0.005168,-0.000000
3742,2023-12-11 16:30:00-05:00,sell,option,AAPL,put,192.33,2023-12-11,0.005199,0.000000,-0.005199,-0.000000
3744,2023-12-11 16:30:00-05:00,sell,option,AAPL,put,193.50,2023-12-10,0.005168,0.310000,-0.005168,-0.001602


In [31]:
def price_lookup_fn(tickers, dt):
    return close_matrix.loc[dt, tickers].values


all_dates = pd.to_datetime(sorted(set([t[0].date() for t in trades])))

run_daily_expiry_loop(
    all_dates=all_dates,
    ledger=ledger,
    close_matrix=close_matrix,
    iv_matrix=vol_data["implied"],
    price_lookup_fn=price_lookup_fn,
    #pnl_tracker=pnl_tracker
)




expiring trades for 2023-11-20 00:00:00
trades_df:                      timestamp action instrument_type underlying option_type  \
0    2023-11-20 00:00:00-05:00   sell          option        AAL        call   
1    2023-11-20 00:00:00-05:00   sell          option        AAP        call   
2    2023-11-20 00:00:00-05:00    buy          option        ABT        call   
3    2023-11-20 00:00:00-05:00    buy          option        ADM        call   
4    2023-11-20 00:00:00-05:00    buy          option        ADP        call   
...                        ...    ...             ...        ...         ...   
4415 2023-12-29 00:00:00-05:00    buy          option        WAB         put   
4416 2023-12-29 00:00:00-05:00   sell          option        WBA         put   
4417 2023-12-29 00:00:00-05:00   sell          option        WBD         put   
4418 2023-12-29 00:00:00-05:00    buy          option        WMB         put   
4419 2023-12-29 00:00:00-05:00   sell          option       ZION     

In [116]:
all_dates=pd.to_datetime(sorted(set([t[0].date() for t in trades])))
get_daily_snapshot(all_dates[0], close_matrix, vol_data["implied"])











,ticker,close,implied_vol
1,AAL,12.39,0.319443
2,AAP,53.04,0.425320
3,AAPL,191.50,0.152162
4,ABBV,138.40,0.154063
6,ABT,101.18,0.161635
...,...,...,...
495,YUM,128.12,0.146756
496,ZBH,112.51,0.230016
497,ZBRA,220.54,0.309009
498,ZION,35.77,0.320162


In [32]:
ledger.trades












,timestamp,action,instrument_type,underlying,option_type,strike,expiry,quantity,price,signed_quantity,total_cost
0,2023-11-20 00:00:00-05:00,sell,option,AAL,call,12.390,2023-11-25,0.080710,0.222394,-0.080710,-0.017949
1,2023-11-20 00:00:00-05:00,buy,option,DD,put,71.380,2023-11-25,0.014010,0.586937,0.014010,0.008223
2,2023-11-20 00:00:00-05:00,sell,option,DISH,put,3.655,2023-11-25,0.273598,0.167796,-0.273598,-0.045908
3,2023-11-20 00:00:00-05:00,sell,option,DXCM,put,109.010,2023-11-25,0.009173,2.194505,-0.009173,-0.020131
4,2023-11-20 00:00:00-05:00,buy,option,ED,put,90.940,2023-11-25,0.010996,0.781653,0.010996,0.008595
...,...,...,...,...,...,...,...,...,...,...,...
8145,2023-12-29 00:00:00-05:00,sell,option,NWL,call,8.680,2024-01-03,0.115207,0.228803,-0.115207,-0.026360
8146,2023-12-29 00:00:00-05:00,sell,option,ON,call,83.560,2024-01-03,0.011967,1.533725,-0.011967,-0.018355
8147,2023-12-29 00:00:00-05:00,sell,option,PARA,call,14.800,2024-01-03,0.067568,0.385509,-0.067568,-0.026048
8148,2023-12-29 00:00:00-05:00,buy,option,KMB,call,121.520,2024-01-03,0.008229,0.908846,0.008229,0.007479


In [45]:
def generate_expiry_report(
    ledger: TradeLedger,
    as_of_date: date,
    price_lookup_fn: Callable[[np.ndarray, date], np.ndarray]
) -> pd.DataFrame:
    """
    Returns a detailed report of positions expired as of 'as_of_date'.
    Includes payoffs, direction, quantity, and total PnL.
    """
    close_dt = datetime.combine(as_of_date, time(16, 30), tzinfo=ZoneInfo("America/New_York"))
    positions = PortfolioView(ledger).positions_at(close_dt)

    # Only include positions whose expiry is today or earlier
    positions["expiry_date"] = pd.to_datetime(positions["expiry"]).dt.date
    to_expire = positions[pd.to_datetime(positions["expiry_date"]) <= pd.to_datetime(as_of_date)].copy()

    if to_expire.empty:
        return pd.DataFrame()  # nothing to report

    # Get spot prices for expired underlyings
    print("about to get spot prices")
    spots = price_lookup_fn(to_expire["underlying"].values, as_of_date)
    print(f"spots: {spots}")

    payoffs = compute_payoff_vec(
        spots,
        to_expire["strike"].values,
        to_expire["option_type"].values
    )

    actions = np.where(to_expire["signed_quantity"] > 0, "sell", "buy")
    abs_qty = to_expire["signed_quantity"].abs().values
    total_pnl = abs_qty * payoffs * np.where(actions == "sell", 1, -1)

    report = pd.DataFrame({
        "date": as_of_date,
        "underlying": to_expire["underlying"].values,
        "option_type": to_expire["option_type"].values,
        "strike": to_expire["strike"].values,
        "expiry": to_expire["expiry"].values,
        "action": actions,
        "quantity": abs_qty,
        "spot": spots,
        "payoff": payoffs,
        "pnl": total_pnl
    })

    return report

In [50]:
expiry_report = generate_expiry_report(ledger, date(2023, 12, 4), price_lookup_fn)
print(expiry_report.head())

trades_df:                       timestamp action instrument_type underlying option_type  \
0     2023-11-20 00:00:00-05:00   sell          option        AAL        call   
1     2023-11-20 00:00:00-05:00   sell          option        AAP        call   
2     2023-11-20 00:00:00-05:00    buy          option        ABT        call   
3     2023-11-20 00:00:00-05:00    buy          option        ADM        call   
4     2023-11-20 00:00:00-05:00    buy          option        ADP        call   
...                         ...    ...             ...        ...         ...   
8145  2023-12-27 16:30:00-05:00   sell          option        WMB         put   
8146  2023-12-27 16:30:00-05:00   sell          option        XYL        call   
8147  2023-12-27 16:30:00-05:00   sell          option        XYL         put   
8148  2023-12-27 16:30:00-05:00    buy          option       ZION        call   
8149  2023-12-27 16:30:00-05:00    buy          option       ZION         put   

       strike   